# Market Basket Analysis

This is my market basket analysis project on a pizza store dataset. This project identifies hidden trends in customer purchasing behavior by analyzing what products are frequently bought together in a pizza store's transaction. The goal of market basket analysis is to identify frequently occurring itemsets or combinations of products to understand customer buying habits.

You can get the dataset through this link